# General setup 
This script takes simulation data directory from SuperSimPy and outputs the data needed for timed tree estimation.
This includes
1. Creating a sampling only times list.
2. Rescaling `sim.tree` to be mutations per site. 


## Sampling times

In [7]:
from ete3 import Tree
from Bio import Phylo
import pandas as pd
import numpy as np
sim_data_path = '/home/joel/EBI/SuperSimPy/data/'
tree = Tree(sim_data_path+'newick_output_tree.nwk')
metadata = pd.read_csv(sim_data_path+'newick_output_metadata.tsv', sep='\t')

In [4]:
leaves = np.array(tree.get_leaves())
namer = np.vectorize(lambda x: x.name)
sample_ids = namer(leaves)

In [5]:
sample_times = metadata[metadata['strain'].isin(sample_ids)][['strain', 'time']]
sample_times.to_csv(sim_data_path + 'datefile.txt', index=False, sep='\t', header=None)

```shell
/home/joel/EBI/lsd2/src/lsd2 -i sim.tree -d datefile.txt -a 0 -s 31101
```

## Tree rescaling 

In [23]:
genome_length = 31101
mut_tree = Phylo.read(sim_data_path+'sim.tree', 'newick')
mut_tree.clade.clades

[Clade(branch_length=2.5393652536720692, comment='&mutations={G12163T,G9910A,A18459G,T19473C,C2838T,T22195C...'),
 Clade(branch_length=0.554599667284565, comment='&mutations={T1552C}')]

In [30]:
def subs_per_site(mutation_comment):
    return (mutation_comment.count(',') + 1)/genome_length

In [33]:
def subs_per_site_tree(root):
    stack = [root.clade]
    while stack:
        node = stack.pop()
        node.branch_length = subs_per_site(node.comment)
        if not node.is_terminal():
            for clade in node.clades:
                stack.append(clade)
    return root

In [39]:
rescaled_tree = subs_per_site_tree(mut_tree)
Phylo.write(rescaled_tree, sim_data_path + 'sim.substitutions.tree', 'newick');